In [ ]:
!pip3 install pyspark

In [ ]:
from pyspark.sql import SparkSession, Row
from pyspark.sql import types as T
from pyspark.sql import window as W
from pyspark.sql import functions as F

spark = SparkSession.builder \
        .master("local") \
        .appName("Colab") \
        .getOrCreate()

## 1. Linear Regression

In [ ]:
df = spark.read.csv("/content/sample_data/california_housing_train.csv", header=True)
df.show()

In [ ]:
df.count(), len(df.columns)

In [ ]:
df.columns[:-1]

In [ ]:
for col in df.columns:
  df = df.withColumn(col, F.col(col).cast(T.DoubleType()))


df.printSchema()

In [ ]:
from pyspark.ml.feature import VectorAssembler

featureassembler = VectorAssembler(inputCols=df.columns[:-1], outputCol='features')

In [ ]:
df_ = featureassembler.transform(df)

In [ ]:
model_df = df_.select('median_house_value','features')
model_df.show(5, truncate=False)

In [ ]:
train_df, test_df = model_df.randomSplit(weights=[0.75, 0.25], seed=42)

In [ ]:
train_df.count(), test_df.count()

In [ ]:
from pyspark.ml.regression import LinearRegression

lg_model = LinearRegression(featuresCol='features', labelCol='median_house_value')

In [ ]:
lg_fit = lg_model.fit(train_df)

lg_fit.coefficients

In [ ]:
pred_train = lg_fit.evaluate(train_df)

pred_train.predictions.show()

In [ ]:
pred_test = lg_fit.evaluate(test_df)

pred_test.predictions.show()

In [ ]:
print("R2 for train data : ", pred_train.r2)
print("R2 for test data : ", pred_test.r2)

## 2. Random Forest

In [ ]:
iris = spark.read.csv('/content/iris.csv', header=True)
iris.show(5)

In [ ]:
for cols in iris.columns[:-1]:
  iris = iris.withColumn(cols, F.col(cols).cast(T.DoubleType()))

iris.printSchema()

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = iris.columns[:-1], outputCol= "features")

In [ ]:
iris_assemble = assembler.transform(iris)

iris_assemble.show()

In [ ]:
iris.groupby('species').count().show()

In [ ]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol = 'species', outputCol='label_index')

In [ ]:
iris_index = indexer.fit(iris_assemble).transform(iris_assemble)

In [ ]:
iris_index.dropDuplicates(subset = ['species', 'label_index']).show()

In [ ]:
iris_train, iris_test = iris_index.randomSplit([0.8, 0.2], seed=42)

In [ ]:
iris_train.count(), iris_test.count()

In [ ]:
iris_train.columns

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

randomforest = RandomForestClassifier(featuresCol='features', labelCol='label_index')

In [ ]:
rf_model = randomforest.fit(iris_test)

pred_test = rf_model.transform(iris_train)

In [ ]:
pred_test.show()

In [ ]:
iris_prediction = pred_test.select('label_index', 'prediction')
iris_prediction.show()

In [ ]:
iris_prediction.printSchema()

In [ ]:
iris_prediction.filter(F.col('label_index') != F.col('prediction')).show()

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol = 'label_index', predictionCol='prediction')

evaluator.evaluate(iris_prediction)

In [ ]:
from pyspark.mllib.evaluation import MulticlassMetrics

metrix = MulticlassMetrics(iris_prediction.rdd.map(tuple))

In [ ]:
metrix.confusionMatrix().toArray()